### **Scrape the Data from the Web**

In [1]:
#for writing to excel(xlsx) we will be needing XlsxWriter, please install it first if you don't have it!
try:
  import XlsxWriter
except ModuleNotFoundError:
  print("XlsxWriter is not installed!!")
  get_ipython().system("pip install XlsxWriter")

#to scrape a table from a webpage
from urllib.parse import urlparse,urlsplit
import requests
import pandas as pd
import os


urls=["https://www.openacessjournal.com/impact-factor-list-journals",
      "https://www.openacessjournal.com/indexed-journals-list",
      "https://www.openacessjournal.com/blog/scopus-indexed-journals/"
      ]


print(len(urls),"Urls Found")

#convert the sheetname- remove _ and - , put title case and remove spaces
def modify_name(my_str):
  replaced=my_str.replace("_", " ").replace("-", " ")
  return replaced.title().replace(" ","")


#get all tables from a url
def get_dataframes(url):
  html = requests.get(url).content
  df_list = pd.read_html(html)
  #print(len(df_list)," Dataframes Returned")
  return df_list

#if df is too small then don't add it
def filter_dfs(dfs_list,min_rows=10):
  new_dfs_list=[]
  for each_df in dfs_list:
    if(len(each_df)>min_rows):
      new_dfs_list.append(each_df)
  return new_dfs_list

#to avoid InvalidWorksheetName: Excel worksheet name 'StatesAndUnionTerritoriesOfIndia1' must be <= 31 chars.
def crop_name(name,thres=29):
  if len(name)<thres:
    return name
  else:
    return name[:thres]

#to get first n elements from list only
def crop_list(lst,thres=29):
  if len(lst)<thres:
    return lst
  else:
    return lst[:thres]

#converts urls to dataframes to excel sheets
#get_max= get the maximum number of tables from each url
#min_rows= the minimum number of rows in each table to save it to the excel sheet
#crop_name_thres= some excel sheets can get quite huge sheet names which blows up the code
#so crop the sheet name for the better purpose

def urls_to_excel(urls,excel_path=None,get_max=10,min_rows=0,crop_name_thres=29):
  excel_path=os.path.join(os.getcwd(),"Excel_Multiple_Sheets_Output.xlsx") if excel_path==None else excel_path
  writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')
  i=0
  for url in urls:
    parsed=urlsplit(url)
    sheet_name=parsed.path.split('/')[-1]
    mod_sheet_name=crop_name(modify_name(sheet_name),thres=crop_name_thres)

    dfs_list=get_dataframes(url)
    filtered_dfs_list=filter_dfs(dfs_list,min_rows=min_rows)
    filtered_dfs_list=crop_list(filtered_dfs_list,thres=get_max)
    for each_df in filtered_dfs_list:
      print("Parsing Excel Sheet "," : ",str(i)+mod_sheet_name)
      i+=1
      each_df.to_excel(writer, sheet_name=str(i)+mod_sheet_name, index=True)
  writer.save()
urls_to_excel(urls,get_max=100,min_rows=1)

XlsxWriter is not installed!!
     |████████████████████████████████| 153kB 3.9MB/s 
3 Urls Found
Parsing Excel Sheet   :  0ImpactFactorListJournals
Parsing Excel Sheet   :  1IndexedJournalsList
Parsing Excel Sheet   :  2
Parsing Excel Sheet   :  3
Parsing Excel Sheet   :  4
Parsing Excel Sheet   :  5
Parsing Excel Sheet   :  6
Parsing Excel Sheet   :  7
Parsing Excel Sheet   :  8
Parsing Excel Sheet   :  9


### Clean the Data Locally

In [2]:
!git clone https://github.com/khanfarhan10/OpenAccessJournalsData.git

Cloning into 'OpenAccessJournalsData'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 2), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [3]:
import os
import pandas as pd
import numpy as np

DATA_FOLDER = "/content/OpenAccessJournalsData/Data"

OAJournals = os.path.join(DATA_FOLDER,"Open_Access_Journals.xlsx")
SCIJournals = os.path.join(DATA_FOLDER,"Scopus_Indexed_Journals.xlsx")

In [4]:
# Read all sheets in your File
dfOA = pd.read_excel(OAJournals, sheet_name=None)
    
# Prints all the sheets name in an ordered dictionary
print(dfOA.keys())

dict_keys(['ImpactFactorListJournals', 'IndexedJournalsList'])


In [5]:
Sheets = list(dfOA.keys())
for sheet_name in Sheets:
  print("Sheet Name :",sheet_name)
  df = dfOA[sheet_name]
  print(df.head())

Sheet Name : ImpactFactorListJournals
   No                            Journals title Impact factor list
0   1                       Academic Pediatrics       2.007 (2020)
1   2                         Acta Orthopaedica       3.217 (2020)
2   3              Advances in Medical Sciences       2.080 (2020)
3   4                 Aesthetic Surgery Journal        3.48 (2020)
4   5  American Journal of Clinical Dermatology       3.840 (2020)
Sheet Name : IndexedJournalsList
   No                          Indexed Journals title
0   1                    Journal of Diabetes Research
1   2                    Journal of Materials Science
2   3                                      The Lancet
3   4              Indian Journal of Medical Research
4   5  Australasian Journal of Educational Technology


In [6]:
if1 = list(dfOA[Sheets[0]]["Impact factor list"])

def cleaner(x):
  try: 
    ans = float(x.split(" ")[0])
  except:
    ans=-1
  return ans

clean_if1 = list(map(lambda x: cleaner(x), if1))

In [7]:
Journals = list(dfOA[Sheets[0]]["Journals title"]) + list(dfOA[Sheets[1]]["Indexed Journals title"])
SourceOA = [Sheets[0]]*len(dfOA[Sheets[0]]["Journals title"]) + [Sheets[1]]*len(dfOA[Sheets[1]]["Indexed Journals title"])
IF2020 = clean_if1 + [-1]*len(dfOA[Sheets[1]]["Indexed Journals title"])

In [8]:
df = pd.DataFrame.from_dict({"JournalName" : Journals,
                            "ImpactFactor2020" : IF2020 ,
                            "Source" : SourceOA    }         
                            )
df["Type"] = "OpenAccess"
# df = pd.DataFrame(columns=["JournalName","ImpactFactor","Source"])
df.head()

,JournalName,ImpactFactor2020,Source,Type
0,Academic Pediatrics,2.007,ImpactFactorListJournals,OpenAccess
1,Acta Orthopaedica,3.217,ImpactFactorListJournals,OpenAccess
2,Advances in Medical Sciences,2.080,ImpactFactorListJournals,OpenAccess
3,Aesthetic Surgery Journal,3.480,ImpactFactorListJournals,OpenAccess
4,American Journal of Clinical Dermatology,3.840,ImpactFactorListJournals,OpenAccess


In [9]:
# Read all sheets in your File
dfSCI = pd.read_excel(SCIJournals, sheet_name=None)
    
# Prints all the sheets name in an ordered dictionary
print(dfSCI.keys())

dict_keys(['SCI_Journals_2020', 'ArtsAndHumanities', 'BusinessManagementAccounting', 'ComputerScience', 'TopCiteScore', 'TopCited', 'TopCitedOpenAccess', 'TopCitations'])


In [10]:
Sheets = list(dfSCI.keys())
All_SCI = []
Source = []
for sheet_name in Sheets:
  each_df = dfSCI[sheet_name]
  current_journals = list(each_df["JournalName"])
  All_SCI = All_SCI + current_journals
  Source = Source+ [sheet_name] * len(current_journals)

In [11]:
len(All_SCI)

702

In [12]:
df2 = pd.DataFrame.from_dict({"JournalName" : All_SCI,
                            "ImpactFactor2020" : [-1]*len(All_SCI) ,
                            "Source" : Source     }         
                            )
df2["Type"] = "SCI"

In [13]:
df2.head()

,JournalName,ImpactFactor2020,Source,Type
0,3 Biotech,-1,SCI_Journals_2020,SCI
1,"3L: Language, Linguistics, Literature",-1,SCI_Journals_2020,SCI
2,AACL Bioflux,-1,SCI_Journals_2020,SCI
3,AAPP Atti della Accademia Peloritana dei Peric...,-1,SCI_Journals_2020,SCI
4,AAPS PharmSciTech,-1,SCI_Journals_2020,SCI


In [14]:
df.head()

,JournalName,ImpactFactor2020,Source,Type
0,Academic Pediatrics,2.007,ImpactFactorListJournals,OpenAccess
1,Acta Orthopaedica,3.217,ImpactFactorListJournals,OpenAccess
2,Advances in Medical Sciences,2.080,ImpactFactorListJournals,OpenAccess
3,Aesthetic Surgery Journal,3.480,ImpactFactorListJournals,OpenAccess
4,American Journal of Clinical Dermatology,3.840,ImpactFactorListJournals,OpenAccess


In [15]:
big_df = df.append(df2)
big_df = big_df.replace(-1,np.nan)

In [16]:
big_df.head()

,JournalName,ImpactFactor2020,Source,Type
0,Academic Pediatrics,2.007,ImpactFactorListJournals,OpenAccess
1,Acta Orthopaedica,3.217,ImpactFactorListJournals,OpenAccess
2,Advances in Medical Sciences,2.080,ImpactFactorListJournals,OpenAccess
3,Aesthetic Surgery Journal,3.480,ImpactFactorListJournals,OpenAccess
4,American Journal of Clinical Dermatology,3.840,ImpactFactorListJournals,OpenAccess


In [17]:
big_df.tail()

,JournalName,ImpactFactor2020,Source,Type
697,Science advances,NaN,TopCitations,SCI
698,The Lancet Global Health,NaN,TopCitations,SCI
699,Light: Science and Applications,NaN,TopCitations,SCI
700,Nucleic Acids Research,NaN,TopCitations,SCI
701,Physical Review X,NaN,TopCitations,SCI


Get ISSN Number and Journal Metrics

In [18]:
!pip install crossrefapi

  Created wheel for crossrefapi: filename=crossrefapi-1.5.0-cp37-none-any.whl size=15720 sha256=34d7bb827fce64a4bb7e4240f2602a691be1b757294ac89a0fa92fe32f5021b9
  Stored in directory: /root/.cache/pip/wheels/73/71/97/9f4adb0f1b5e0406d6b0b52c6e1418ed1aadf4565ca763604a
Successfully built crossrefapi


In [19]:
from crossref.restful import Journals
journals = Journals()

In [20]:
journals.count()

91965

```
Example:
 |          >>> from crossref.restful import Journals
 |          >>> journals = Journals().query('Public Health', 'Health Science')
 |          >>> journals.url
 |          'https://api.crossref.org/journals?query=Public+Health+Health+Science'
 |          >>> next(iter(journals))
 |          {'last-status-check-time': None, 'counts': None, 'coverage': None,
 |          'publisher': 'ScopeMed International Medical Journal Managment and Indexing System',
 |          'flags': None, 'breakdowns': None, 'ISSN': ['2320-4664', '2277-338X'],
 |          'title': 'International Journal of Medical Science and Public Health'}
```

In [21]:
help(journals)

Help on Journals in module crossref.restful object:

class Journals(Endpoint)
 |  Journals(request_url=None, request_params=None, context=None, etiquette=None, throttle=True, crossref_plus_token=None)
 |  
 |  Method resolution order:
 |      Journals
 |      Endpoint
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  journal(self, issn, only_message=True)
 |      This method retrieve a json with the given ISSN metadata
 |      
 |      args: Journal ISSN (String)
 |      
 |      return: Journal JSON data
 |      
 |      Example:
 |          >>> from crossref.restful import Journals
 |          >>> journals = Journals()
 |          >>> journals.journal('2277-338X')
 |          {'last-status-check-time': None, 'counts': None, 'coverage': None,
 |          'publisher': 'ScopeMed International Medical Journal Managment and Indexing System',
 |          'flags': None, 'breakdowns': None, 'ISSN': ['2320-4664', '2277-338X'],
 |          'title': 'International Journal of Medic

In [22]:
search_journals = journals.query('Academic Pediatrics')
resp = next(iter(search_journals))

In [23]:
search_journals.url

'https://api.crossref.org/journals?query=Academic+Pediatrics'

In [24]:
resp

{'ISSN': ['2474-7521', '2474-7521'],
 'breakdowns': {'dois-by-issued-year': [[2016, 46],
   [2017, 41],
   [2018, 37],
   [2019, 34],
   [2020, 33],
   [2015, 2],
   [2021, 1]]},
 'counts': {'backfile-dois': 126, 'current-dois': 68, 'total-dois': 194},
 'coverage': {'abstracts-backfile': 0.0,
  'abstracts-current': 0.0,
  'affiliations-backfile': 0.0,
  'affiliations-current': 0.0,
  'award-numbers-backfile': 0.0,
  'award-numbers-current': 0.0,
  'funders-backfile': 0.0,
  'funders-current': 0.0,
  'licenses-backfile': 0.0,
  'licenses-current': 0.0,
  'open-references-backfile': 0.0,
  'open-references-current': 0.0,
  'orcids-backfile': 0.0,
  'orcids-current': 0.0,
  'references-backfile': 0.0,
  'references-current': 0.0,
  'resource-links-backfile': 0.0,
  'resource-links-current': 0.0,
  'similarity-checking-backfile': 0.0,
  'similarity-checking-current': 0.0,
  'update-policies-backfile': 0.0,
  'update-policies-current': 0.0},
 'coverage-type': {'all': {'abstracts': 0.0,
   '

### Interests
```
'ISSN': ['2474-7521', '2474-7521'],
 'breakdowns': {'dois-by-issued-year': [[2016, 46],
   [2017, 41],
   [2018, 37],
   [2019, 34],
   [2020, 33],
   [2015, 2],
   [2021, 1]]},
 'counts': {'backfile-dois': 126, 'current-dois': 68, 'total-dois': 194}
'publisher': 'Juniper Publishers',
 'subjects': [],
 'title': 'Academic Journal of Pediatrics & Neonatology'
 ```

In [25]:
import json

with open('data.json', 'w') as fp:
    json.dump(resp, fp)

In [26]:
def create_dir(dir, v=1):
    """
    Creates a directory without throwing an error if directory already exists.
    dir : The directory to be created.
    v : Verbosity
    """
    if not os.path.exists(dir):
        os.makedirs(dir)
        if v:
            print("Created Directory : ", dir)
        return 1
    else:
        if v:
            print("Directory already existed : ", dir)
        return 0

Journals_Metadata_Path = "Journals_Metadata"
create_dir(Journals_Metadata_Path)

from copy import deepcopy

def extract_data(attrs,resp):
  try:
    if len(attrs)==1:
      return resp[attrs[0]]
    resp_new= deepcopy(resp)
    for att in attrs:
      resp_new = resp_new[att]
    return resp_new
  except:
    return np.nan

def calc(data):
  res=0
  try:
    for year,val in data:
      res+=val
  except:
    pass
  return res

def load_JSON(fpath):
  with open(fpath) as json_file:
    data = json.load(json_file)
  return data

def get_details(journal_name):
  JSON_FILE_PATH = os.path.join(Journals_Metadata_Path,journal_name+'.json')
  resp={}
  if os.path.exists(JSON_FILE_PATH):
    resp = load_JSON(JSON_FILE_PATH)
  if resp=={}:
    try:
      search_journals = journals.query(journal_name)
      resp = next(iter(search_journals))
    except:
      resp={}
      pass
    create_dir(os.path.dirname(JSON_FILE_PATH))
    with open(JSON_FILE_PATH, 'w') as fp:
      json.dump(resp, fp)
  """
  Data = {"Title": resp['title'],
          "Publisher": resp['publisher'],
          "ISSN" : resp['ISSN'],
          "DoIsbyYear": resp['breakdowns']['dois-by-issued-year'],
          "TotalDoIs": calc(resp['breakdowns']['dois-by-issued-year']),
          "BackFileDoIs": resp['counts']['backfile-dois'],
          "CurrentDoIs" : resp['counts']['current-dois'],
          "TotalDoIs" : resp['counts']['total-dois'],
          'Subjects' : resp['subjects']
          }
  """
  Data = {"Title": extract_data(['title'],resp),
          "Publisher": extract_data(['publisher'],resp),
          "ISSN" : extract_data(['ISSN'],resp),
          "DoIsbyYear": extract_data(['breakdowns','dois-by-issued-year'],resp),
          "TotalDoIs": calc(extract_data(['breakdowns','dois-by-issued-year'],resp)),
          "BackFileDoIs": extract_data(['counts','backfile-dois'],resp),
          "CurrentDoIs" : extract_data(['counts','current-dois'],resp),
          "TotalDoIs" : extract_data(['counts','total-dois'],resp),
          'Subjects' : extract_data(['subjects'],resp),
          'JSON_FILE_PATH':JSON_FILE_PATH
          }
  return Data

get_details(journal_name="The Lancet Global Health")

Created Directory :  Journals_Metadata
Directory already existed :  Journals_Metadata


{'BackFileDoIs': 1707,
 'CurrentDoIs': 1052,
 'DoIsbyYear': [[2019, 435],
  [2020, 399],
  [2018, 390],
  [2017, 357],
  [2016, 308],
  [2014, 280],
  [2015, 272],
  [2021, 218],
  [2013, 100]],
 'ISSN': ['2214-109X'],
 'JSON_FILE_PATH': 'Journals_Metadata/The Lancet Global Health.json',
 'Publisher': 'Elsevier ',
 'Subjects': [{'ASJC': 2700, 'name': 'General Medicine'}],
 'Title': 'The Lancet Global Health',
 'TotalDoIs': 2759}

In [27]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=1024x768")
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

def ask_google(query):

    # Search for query
    query = query.replace(' ', '+')

    wd.get('http://www.google.com/search?q=' + query)

    # Get text from Google answer box

    answer = wd.execute_script(
            "return document.elementFromPoint(arguments[0], arguments[1]);",
            350, 230).text

    return answer

     |████████████████████████████████| 911kB 3.9MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ P

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


In [28]:
ask_google(query="impact factor Academic Journal of Pediatrics & Neonatology")

'5.417'

In [29]:
import re
def get_IF(journal_name):
  query = f"impact factor {journal_name}"
  res = ask_google(query)
  # print(res)
  try:
    res = float(res)
  except:
    new_res = re.findall(r"[-+]?\d*\.?\d+|\d+", res)
    # print(new_res)
    for res in new_res:
      if float(res)<100 and float(res)>0:
        return float(res)
    res = np.nan
    pass
  return res

get_IF("The Lancet Global Health")

21.597

In [30]:
big_df.head()

,JournalName,ImpactFactor2020,Source,Type
0,Academic Pediatrics,2.007,ImpactFactorListJournals,OpenAccess
1,Acta Orthopaedica,3.217,ImpactFactorListJournals,OpenAccess
2,Advances in Medical Sciences,2.080,ImpactFactorListJournals,OpenAccess
3,Aesthetic Surgery Journal,3.480,ImpactFactorListJournals,OpenAccess
4,American Journal of Clinical Dermatology,3.840,ImpactFactorListJournals,OpenAccess


```
Data = {"Title": extract_data(['title'],resp),
          "Publisher": extract_data(['publisher'],resp),
          "ISSN" : extract_data(['ISSN'],resp),
          "DoIsbyYear": extract_data(['breakdowns','dois-by-issued-year'],resp),
          "TotalDoIs": calc(extract_data(['breakdowns','dois-by-issued-year'],resp)),
          "BackFileDoIs": extract_data(['counts','backfile-dois'],resp),
          "CurrentDoIs" : extract_data(['counts','current-dois'],resp),
          "TotalDoIs" : extract_data(['counts','total-dois'],resp),
          'Subjects' : extract_data(['subjects'],resp)
          }
```

Title, Publisher, ISSN, DoIsbyYear, TotalDoIs, BackFileDoIs, CurrentDoIs, Subjects, IF


In [31]:
Title = []
Publisher = []
ISSN = []
DoIsbyYear = []
BackFileDoIs = []
CurrentDoIs = []
TotalDoIs = []
Subjects = []
IF = []
Journals_Metadata_Paths = []



import time
"""time.sleep(5)""" # perform per 50 or 100 iters
import tqdm
for index, row in tqdm.tqdm(big_df.iterrows()):
  if index%50==0:
    time.sleep(5) 
  jname = row["JournalName"]
  Data = get_details(journal_name=jname)
  impact_factor = np.nan
  # print(Data)
  if not Data["Title"]==np.nan:
    impact_factor = get_IF(Data["Title"])
  if impact_factor==np.nan:
    impact_factor = get_IF(jname)
  Title.append(Data["Title"])
  Publisher.append(Data["Publisher"])
  ISSN.append(Data["ISSN"])
  DoIsbyYear.append(Data["DoIsbyYear"])
  BackFileDoIs.append(Data["BackFileDoIs"])
  CurrentDoIs.append(Data["CurrentDoIs"])
  TotalDoIs.append(Data["TotalDoIs"])
  Subjects.append(Data["Subjects"])
  IF.append(impact_factor)
  Journals_Metadata_Paths.append(Data["JSON_FILE_PATH"])


0it [00:00, ?it/s]

Directory already existed :  Journals_Metadata


1it [00:05,  5.52s/it]

Directory already existed :  Journals_Metadata


2it [00:06,  4.16s/it]

...

1067it [19:42,  1.00s/it]

Directory already existed :  Journals_Metadata


1069it [19:49,  2.47s/it]

Directory already existed :  Journals_Metadata


1070it [19:50,  1.11s/it]


In [35]:
"""1071 iters"""
AllJournalNames = big_df["JournalName"]
ImpactFactor2020 = big_df["ImpactFactor2020"]
Sources = big_df["Source"]
Types = big_df["Type"]

In [36]:
big_df.to_csv("OpenAccessJournalsDataRaw.csv",index=False)

In [38]:
big_df.to_excel("OpenAccessJournalsDataRaw.xlsx",index=False)

In [37]:
main_df = pd.DataFrame.from_dict({"JournalName" : AllJournalNames,
                            "ImpactFactor2020" : ImpactFactor2020 ,
                            "Source" : Sources,
                            "Type" : Types,
                            "Title" : Title,
                            "Publisher":Publisher,
                            "ISSN":ISSN,
                            "DoIsbyYear":DoIsbyYear,
                            "BackFileDoIs":BackFileDoIs ,
                            "CurrentDoIs":CurrentDoIs,
                            "TotalDoIs":TotalDoIs,
                            "Subjects":Subjects,
                            "ImpactFactor":IF,
                            "Journals_Metadata_Paths":Journals_Metadata_Paths
                            })
main_df.to_csv("MainOpenAccessJournalsData.csv",index=False)

In [39]:
main_df.to_excel("MainOpenAccessJournalsData.xlsx",index=False)